# Nasa NEX with Kerchunk

In [1]:
import pandas as pd 
import xarray as xr 
import s3fs
import hvplot.xarray
import glob
import logging
from tempfile import TemporaryDirectory
from fsspec.implementations.reference import LazyReferenceMapper, ReferenceFileSystem
import os 
import dask
import fsspec
import s3fs
import ujson
from distributed import Client
from kerchunk.combine import MultiZarrToZarr
from kerchunk.hdf import SingleHdf5ToZarr



In [20]:
## loading
df = pd.read_csv(
    "s3://carbonplan-climate-impacts/extreme-heat/v1.0/inputs/nex-gddp-cmip6-files.csv"
)
str_split_urls = df[" fileURL"].str.rsplit('/',expand=True) 
edf = pd.DataFrame()
edf['url'] = df[" fileURL"].replace({'https://nex-gddp-cmip6.s3.us-west-2.amazonaws.com/':'s3://nex-gddp-cmip6/'},regex=True)
edf['GCM'] = str_split_urls[4]
edf['scenario'] = str_split_urls[5]
edf['ensemble_member'] = str_split_urls[6]
edf['variable'] = str_split_urls[7]
# str strip to remove all whitespace
edf_obj = edf.select_dtypes(['object'])
edf[edf_obj.columns] = edf_obj.apply(lambda x: x.str.strip())



In [23]:
edf.to_csv('nasa_nex_formatted.csv',index=False)

In [3]:
file_pattern = edf.query("GCM == 'ACCESS-CM2'  & scenario == 'historical' & ensemble_member == 'r1i1p1f1'")['url']
# about 53 GB in netcdf file size!

In [ ]:
fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False)
ds = xr.open_dataset(fs.open(edf['url'].iloc[0]),engine='h5netcdf')

In [4]:
client = Client(n_workers=8, silence_logs=logging.ERROR)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:64373,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:64393,Total threads: 1
Dashboard: http://127.0.0.1:64399/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:64376,


2023-09-07 11:22:52,670 - tornado.application - ERROR - Exception in callback functools.partial(<function TCPServer._handle_connection.<locals>.<lambda> at 0x1525aeb00>, <Task finished name='Task-4584' coro=<BaseTCPListener._handle_stream() done, defined at /Users/nrhagen/micromamba/envs/nasa-nex/lib/python3.10/site-packages/distributed/comm/tcp.py:598> exception=MemoryError((6073139484287059271,), dtype('uint8'))>)
Traceback (most recent call last):
  File "/Users/nrhagen/micromamba/envs/nasa-nex/lib/python3.10/site-packages/tornado/ioloop.py", line 738, in _run_callback
    ret = callback()
  File "/Users/nrhagen/micromamba/envs/nasa-nex/lib/python3.10/site-packages/tornado/tcpserver.py", line 387, in <lambda>
    gen.convert_yielded(future), lambda f: f.result()
  File "/Users/nrhagen/micromamba/envs/nasa-nex/lib/python3.10/site-packages/distributed/comm/tcp.py", line 610, in _handle_stream
    await self.on_connection(comm)
  File "/Users/nrhagen/micromamba/envs/nasa-nex/lib/python

In [5]:
file_pattern = file_pattern[0:8]

In [7]:

fs_read = fsspec.filesystem("s3", anon=True, skip_instance_cache=True)
# This dictionary will be passed as kwargs to `fsspec`. For more details, check out the `foundations/kerchunk_basics` notebook.
so = dict(mode="rb", anon=True, default_fill_cache=False, default_cache_type="first")

# We are creating a temporary directory to store the .json reference files
# Alternately, you could write these to cloud storage.
td = TemporaryDirectory()
temp_dir = td.name
temp_dir

'/var/folders/mb/7d7yq_4j2qgdfm_j3j4tsyl40000gn/T/tmpdo0_gu_w'

In [8]:
# Use Kerchunk's `SingleHdf5ToZarr` method to create a `Kerchunk` index from a NetCDF file.
def generate_json_reference(fil, output_dir: str):
    with fs_read.open(fil, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, fil, inline_threshold=300)
        fname = fil.split("/")[-1].strip(".nc")
        outf = f"{output_dir}/{fname}.json"
        with open(outf, "wb") as f:
            f.write(ujson.dumps(h5chunks.translate()).encode())
        return outf


# Generate Dask Delayed objects
tasks = [dask.delayed(generate_json_reference)(fil, temp_dir) for fil in file_pattern]

In [9]:
# Start parallel processing
import warnings

warnings.filterwarnings("ignore")
dask.compute(tasks)

(['/var/folders/mb/7d7yq_4j2qgdfm_j3j4tsyl40000gn/T/tmpdo0_gu_w/hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1950.json',
  '/var/folders/mb/7d7yq_4j2qgdfm_j3j4tsyl40000gn/T/tmpdo0_gu_w/hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1951.json',
  '/var/folders/mb/7d7yq_4j2qgdfm_j3j4tsyl40000gn/T/tmpdo0_gu_w/hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1952.json',
  '/var/folders/mb/7d7yq_4j2qgdfm_j3j4tsyl40000gn/T/tmpdo0_gu_w/hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1953.json',
  '/var/folders/mb/7d7yq_4j2qgdfm_j3j4tsyl40000gn/T/tmpdo0_gu_w/hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1954.json',
  '/var/folders/mb/7d7yq_4j2qgdfm_j3j4tsyl40000gn/T/tmpdo0_gu_w/hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1955.json',
  '/var/folders/mb/7d7yq_4j2qgdfm_j3j4tsyl40000gn/T/tmpdo0_gu_w/hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1956.json',
  '/var/folders/mb/7d7yq_4j2qgdfm_j3j4tsyl40000gn/T/tmpdo0_gu_w/hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1957.json'],)

In [10]:
output_files = glob.glob(f"{temp_dir}/*.json")

len(output_files)

8

In [11]:
fs = fsspec.filesystem("file")

if os.path.exists("combined.parq"):
    import shutil

    shutil.rmtree("combined.parq")
os.makedirs("combined.parq")

out = LazyReferenceMapper.create(1000, "combined.parq", fs)

mzz = MultiZarrToZarr(
    output_files,
    remote_protocol="memory",
    concat_dims=["time"],
    identical_dims=["lat", "lon"],
    out=out,
).translate()

out.flush()

In [5]:
fs = ReferenceFileSystem(
    "combined.parq", remote_protocol="s3", target_protocol="file", lazy=True
)
ds = xr.open_dataset(
    fs.get_mapper(), engine="zarr", backend_kwargs={"consolidated": False}, chunks={}
)


In [6]:
ds

<xarray.Dataset>
Dimensions:  (time: 2922, lat: 600, lon: 1440)
Coordinates:
  * lat      (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon      (lon) float64 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * time     (time) datetime64[ns] 1950-01-01T12:00:00 ... 1957-12-31T12:00:00
Data variables:
    hurs     (time, lat, lon) float32 dask.array<chunksize=(1, 600, 1440), meta=np.ndarray>
Attributes: (12/22)
    Conventions:           CF-1.7
    activity:              NEX-GDDP-CMIP6
    cmip6_institution_id:  CSIRO-ARCCSS
    cmip6_license:         CC-BY-SA 4.0
    cmip6_source_id:       ACCESS-CM2
    contact:               Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bridget...
    ...                    ...
    scenario:              historical
    source:                BCSD
    title:                 ACCESS-CM2, r1i1p1f1, historical, global downscale...
    tracking_id:           16d27564-470f-41ea-8077-f4cc3efa5bfe
    variant_label:         r1i1p1f1
    version:               1.0

In [ ]:
# JSON 
# # Create a list of reference json files
# output_files = glob.glob(f"{temp_dir}/*.json")

# # combine individual references into single consolidated reference
# mzz = MultiZarrToZarr(
#     output_files,
#     concat_dims=["time"],
#     identical_dims=["lat", "lon"],
# )
# # save translate reference in memory for later visualization
# multi_kerchunk = mzz.translate()

# # Write kerchunk .json record

# output_fname = "references/ACCESS-CM2_historical_r1i1p1f1.json"
# with open(f"{output_fname}", "wb") as f:
#     f.write(ujson.dumps(multi_kerchunk).encode())

In [ ]:
# create an fsspec reference filesystem from the Kerchunk output
import fsspec

fs = fsspec.filesystem(
    "reference",
    fo="references/ACCESS-CM2_historical_r1i1p1f1.json",
    remote_protocol="s3",
    remote_options={"anon": True},
    skip_instance_cache=True,
)
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", chunks={})

In [19]:
ds.chunks

Frozen({})

In [17]:
display(ds.hvplot("lon", "lat", rasterizes=True))


BokehModel(combine_events=True, render_bundle={'docs_json': {'696b8aa6-d6e0-4419-a6dd-aae463137890': {'version…